# Model 3 — Parts & Technician Recommendation (Apriori + Cross-Domain)
This notebook builds an association-rule-based recommender for **(part, technician_group)** given **(fault_category, device_type)**.  
It supports:
- Preprocessing & column mapping
- Apriori rule mining (support, confidence, lift)
- Heuristic layer (min-confidence & min-lift)
- Ranking (support × confidence)
- Evaluation (precision@k, recall@k)
- Cross-domain transfer: train on MaintNet (vehicle/facility), validate on aviation
- Optional: boost weights using Scania-derived clusters

> **Files expected** (adjust paths if needed):
- `/content/merged_maintnet_dataset.csv`
- `/content/Facility_Maintenance_Dataset.csv`
- `/content/df_scania_clean.csv`

In [ ]:
!pip -q install mlxtend pandas numpy scikit-learn

import pandas as pd
import numpy as np
from collections import defaultdict
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option("display.max_colwidth", 120)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Configuration — paths, mappings, and simple NLP extractors

In [ ]:
# === File paths (Colab) ===
PATH_MERGED = "content/merged_maintnet_dataset.csv"
PATH_FACILITY = "content/Facility_Maintenance_Dataset.csv"
PATH_SCANIA = "content/df_scania_clean.csv"

# === Column mapping ===
# Map *your* dataset fields to the canonical names used in this notebook.
# If a field is missing, leave as None and we'll try to extract from text (e.g., DESCRIPTION).
CONFIG = {
    "fault_category_col": None,        # e.g., "PROB_TYPE"
    "device_type_col": None,           # e.g., derived from DESCRIPTION
    "part_col": None,                  # e.g., derived from DESCRIPTION
    "technician_group_col": None,      # e.g., derived from DESCRIPTION or rules
    "free_text_col": "DESCRIPTION",    # column to mine if above are None
    "domain_col": None,                # if you have a column marking 'vehicle'/'facility'/'aviation'
}

# === Domain rules (edit to match your data) ===
def infer_domain(row: pd.Series) -> str:
    """Return one of {'vehicle','facility','aviation'} based on available fields.
    Customize as needed for your data.
    """
    txt = ""
    for c in [CONFIG.get("free_text_col"), CONFIG.get("fault_category_col"), CONFIG.get("device_type_col")]:
        if c and pd.notna(row.get(c, None)):
            txt += f" {str(row[c]).lower()}"
    # Simple heuristics — replace with your real rules
    if "plane" in txt or "aircraft" in txt or "hangar" in txt or "aviation" in txt:
        return "aviation"
    if "vehicle" in txt or "truck" in txt or "bus" in txt:
        return "vehicle"
    return "facility"

# === Simple keyword dictionaries (edit/extend!) ===
DEVICE_LEXICON = {
    "hvac": ["hvac", "air handler", "duct", "vent", "thermostat"],
    "electrical": ["switch", "breaker", "panel", "wiring", "socket", "outlet", "power"],
    "plumbing": ["pipe", "leak", "valve", "drain", "toilet", "sink", "pump"],
    "mechanical": ["motor", "gear", "bearing", "belt", "compressor"],
}

PART_LEXICON = {
    "valve": ["valve", "solenoid"],
    "gasket": ["gasket", "seal", "o-ring", "o ring"],
    "filter": ["filter"],
    "pump": ["pump", "impeller"],
    "fuse": ["fuse"],
    "thermostat": ["thermostat"],
}

TECH_GROUP_RULES = {
    "HVAC": ["hvac", "air handler", "duct", "vent", "thermostat", "compressor", "filter"],
    "ELECTRICAL": ["switch", "breaker", "panel", "wiring", "socket", "outlet", "fuse", "power"],
    "PLUMBING": ["pipe", "leak", "valve", "drain", "toilet", "sink", "pump"],
    "MECHANICAL": ["motor", "gear", "bearing", "belt", "compressor", "gasket"],
}

def extract_from_text(text: str, lexicon: dict, default: str = "unknown"):
    if not isinstance(text, str):
        return default
    low = text.lower()
    for label, keys in lexicon.items():
        if any(k in low for k in keys):
            return label
    return default

def map_technician_group(text: str) -> str:
    if not isinstance(text, str):
        return "GENERAL"
    low = text.lower()
    for group, keys in TECH_GROUP_RULES.items():
        if any(k in low for k in keys):
            return group
    return "GENERAL"

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Load datasets

In [ ]:

# def try_load(path):
#     try:
#         return pd.read_csv(path)
#     except Exception as e:
#         print(f"Could not read {path}: {e}")
#         return None

# df_maint = try_load(PATH_FACILITY)  # facility-style maintenance logs
# df_merged = try_load(PATH_MERGED)   # optional: vocab/auxiliary
# df_scania = try_load(PATH_SCANIA)   # optional: Scania clusters

# print("Rows — Facility:", 0 if df_maint is None else len(df_maint))
# print("Rows — Merged:", 0 if df_merged is None else len(df_merged))
# print("Rows — Scania:", 0 if df_scania is None else len(df_scania))

# # If you have multiple MaintNet sources, you can concat them here.
# maint_sources = [d for d in [df_maint] if d is not None]
# df = pd.concat(maint_sources, ignore_index=True) if maint_sources else pd.DataFrame()

# print("Combined MaintNet rows:", len(df))
# print("Columns:", list(df.columns)[:30])
# df.head(3)




/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Build canonical columns: fault_category, device_type, part, technician_group, domain

In [ ]:

def build_canonical(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    df = df.copy()

    free_txt = CONFIG.get("free_text_col")
    # Fault category
    if CONFIG.get("fault_category_col") and CONFIG["fault_category_col"] in df.columns:
        df["fault_category"] = df[CONFIG["fault_category_col"]].astype(str).str.lower()
    else:
        # naïve: use coarse category from text if PROB_TYPE not present
        if free_txt and free_txt in df.columns:
            df["fault_category"] = df[free_txt].astype(str).str.extract(r'(?i)(leak|overheat|no power|vibration|noise|smell)', expand=False).str.lower()
            df["fault_category"] = df["fault_category"].fillna("general")
        else:
            df["fault_category"] = "general"

    # Device type
    if CONFIG.get("device_type_col") and CONFIG["device_type_col"] in df.columns:
        df["device_type"] = df[CONFIG["device_type_col"]].astype(str).str.lower()
    else:
        if free_txt and free_txt in df.columns:
            df["device_type"] = df[free_txt].apply(lambda x: extract_from_text(x, DEVICE_LEXICON, default="unknown")).str.lower()
        else:
            df["device_type"] = "unknown"

    # Part
    if CONFIG.get("part_col") and CONFIG["part_col"] in df.columns:
        df["part"] = df[CONFIG["part_col"]].astype(str).str.lower()
    else:
        if free_txt and free_txt in df.columns:
            df["part"] = df[free_txt].apply(lambda x: extract_from_text(x, PART_LEXICON, default="unknown")).str.lower()
        else:
            df["part"] = "unknown"

    # Technician Group
    if CONFIG.get("technician_group_col") and CONFIG["technician_group_col"] in df.columns:
        df["technician_group"] = df[CONFIG["technician_group_col"]].astype(str).str.upper()
    else:
        if free_txt and free_txt in df.columns:
            df["technician_group"] = df[free_txt].apply(map_technician_group).str.upper()
        else:
            df["technician_group"] = "GENERAL"

    # Domain
    if CONFIG.get("domain_col") and CONFIG["domain_col"] in df.columns:
        df["domain"] = df[CONFIG["domain_col"]].astype(str).str.lower()
    else:
        df["domain"] = df.apply(infer_domain, axis=1).str.lower()

    # Keep only needed columns
    keep = ["fault_category","device_type","part","technician_group","domain"]
    return df[keep]

df_canon = build_canonical(df)
df_canon.head(10)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

NameError: name 'df' is not defined

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Cross-domain split (train on vehicle/facility, test on aviation)

In [ ]:

def cross_domain_split(df_canon: pd.DataFrame):
    if df_canon.empty:
        return df_canon, df_canon
    df_canon = df_canon.copy()
    train = df_canon[df_canon["domain"].isin(["vehicle","facility"])]
    test  = df_canon[df_canon["domain"].isin(["aviation"])]
    # If no aviation present, fall back to random 80/20 split so you can still test the code path
    if test.empty:
        rnd = df_canon.sample(frac=1.0, random_state=42).reset_index(drop=True)
        cut = int(0.8 * len(rnd))
        train, test = rnd.iloc[:cut].copy(), rnd.iloc[cut:].copy()
        print("⚠️ No 'aviation' rows found — using random 80/20 split for demonstration.")
    print(f"Train size: {len(train)}, Test size: {len(test)}")
    return train, test

train_df, test_df = cross_domain_split(df_canon)
train_df.head(5)


## Apriori rule mining — build transactions and mine rules

In [ ]:

def build_transaction_frame(df: pd.DataFrame) -> pd.DataFrame:
    """Create one-hot item matrix with prefixed tokens:
    Antecedent candidates: F_{fault_category}, D_{device_type}
    Consequent candidates: P_{part}, T_{technician_group}
    """
    items = {
        "F": "fault_category",
        "D": "device_type",
        "P": "part",
        "T": "technician_group",
    }
    rows = []
    for _, r in df.iterrows():
        tokens = []
        for pref, col in items.items():
            v = str(r[col]).strip().lower()
            if v and v != "nan":
                tokens.append(f"{pref}_{v}")
        rows.append(tokens)
    # One-hot encode
    uniq = sorted({t for row in rows for t in row})
    mat = np.zeros((len(rows), len(uniq)), dtype=np.uint8)
    idx = {u:i for i,u in enumerate(uniq)}
    for i,row in enumerate(rows):
        for t in row:
            mat[i, idx[t]] = 1
    ohe = pd.DataFrame(mat, columns=uniq)
    return ohe

def mine_rules(ohe: pd.DataFrame, min_support=0.005, min_confidence=0.2, min_lift=1.0):
    freq = apriori(ohe, min_support=min_support, use_colnames=True)
    rules = association_rules(freq, metric="confidence", min_threshold=min_confidence)
    # Heuristic filter: antecedents must be only F_/D_, consequents only P_/T_
    def is_ante_ok(s): return all(str(x).startswith(("F_","D_")) for x in s)
    def is_cons_ok(s):  return all(str(x).startswith(("P_","T_")) for x in s)
    mask = rules["antecedents"].apply(is_ante_ok) & rules["consequents"].apply(is_cons_ok)
    rules = rules[mask].copy()
    # Apply lift threshold
    rules = rules[rules["lift"] >= min_lift].copy()
    # Score for ranking
    rules["score"] = rules["support"] * rules["confidence"]
    # Convenience columns
    rules["ante_str"] = rules["antecedents"].apply(lambda s: tuple(sorted([str(x) for x in s])))
    rules["cons_str"]  = rules["consequents"].apply(lambda s: tuple(sorted([str(x) for x in s])))
    rules = rules.sort_values(["score","lift","confidence"], ascending=False).reset_index(drop=True)
    return rules

ohe_train = build_transaction_frame(train_df)
rules = mine_rules(
    ohe_train,
    min_support=0.005,       # tune
    min_confidence=0.2,      # tune
    min_lift=1.0             # tune
)
print(f"Mined {len(rules)} rules after filtering.")
rules.head(10)[["ante_str","cons_str","support","confidence","lift","score"]]


## Inference — generate top‑k recommendations

In [ ]:

from typing import List, Tuple

def match_rules_for_context(rules: pd.DataFrame, fault_category: str, device_type: str) -> pd.DataFrame:
    a = tuple(sorted([f"F_{str(fault_category).lower()}", f"D_{str(device_type).lower()}"]))
    # Accept rules whose antecedents are a subset of the context (allowing F-only or D-only rules)
    ctx = set(a)
    return rules[rules["antecedents"].apply(lambda s: set(map(str, s)).issubset(ctx))].copy()

def extract_recos(rules_sub: pd.DataFrame, top_k=5) -> Tuple[List[str], List[str]]:
    """Return (parts, technicians) lists sorted by rule score (deduped)."""
    parts, techs = [], []
    seen_p, seen_t = set(), set()
    for _, row in rules_sub.sort_values("score", ascending=False).iterrows():
        for c in row["consequents"]:
            c = str(c)
            if c.startswith("P_"):
                p = c[2:]
                if p not in seen_p:
                    parts.append(p); seen_p.add(p)
            elif c.startswith("T_"):
                t = c[2:]
                if t not in seen_t:
                    techs.append(t); seen_t.add(t)
        if len(parts) >= top_k and len(techs) >= top_k:
            break
    return parts[:top_k], techs[:top_k]

def recommend_for_row(row: pd.Series, rules: pd.DataFrame, top_k=5):
    sub = match_rules_for_context(rules, row["fault_category"], row["device_type"])
    return extract_recos(sub, top_k=top_k)

# Demo on a few rows
for i in range(min(3, len(test_df))):
    parts_k, techs_k = recommend_for_row(test_df.iloc[i], rules, top_k=5)
    print(f"Context[{i}] = (F={test_df.iloc[i]['fault_category']} , D={test_df.iloc[i]['device_type']})")
    print("  parts  ->", parts_k)
    print("  techs  ->", techs_k)


## Evaluation — precision@k and recall@k

In [ ]:

def precision_recall_at_k(test_df: pd.DataFrame, rules: pd.DataFrame, ks=(1,3,5)):
    metrics = {k: {"prec": [], "rec": []} for k in ks}
    for _, row in test_df.iterrows():
        true_parts = {str(row["part"]).lower()}
        true_techs = {str(row["technician_group"]).upper()}
        for k in ks:
            parts_k, techs_k = recommend_for_row(row, rules, top_k=k)
            # Treat it as two independent recommendation tasks and average
            hit_p = 1 if any(p in true_parts for p in parts_k) else 0
            hit_t = 1 if any(t.upper() in true_techs for t in techs_k) else 0
            # For single-label truth, precision@k is hits/k for each list; recall@k == hits/1
            prec = ((hit_p / max(1, k)) + (hit_t / max(1, k))) / 2.0
            rec  = ((hit_p / 1.0) + (hit_t / 1.0)) / 2.0
            metrics[k]["prec"].append(prec)
            metrics[k]["rec"].append(rec)
    out = []
    for k in ks:
        p = np.mean(metrics[k]["prec"]) if metrics[k]["prec"] else 0.0
        r = np.mean(metrics[k]["rec"]) if metrics[k]["rec"] else 0.0
        out.append({"k": k, "precision@k": round(p,4), "recall@k": round(r,4)})
    return pd.DataFrame(out)

eval_df = precision_recall_at_k(test_df, rules, ks=(1,3,5))
eval_df


## Optional — Scania-derived clusters to reweight rules

In [ ]:

# This is a placeholder: if you have true fault clusters from Scania (e.g., k-means labels),
# map cluster IDs to fault categories or device types and upweight rules that align.
# Here, we show a trivial example that upweights rules when 'F_overheat' is present
# and Scania class_label == 1 (pretend: 'overheat-like' failures).

def apply_scania_weighting(rules: pd.DataFrame, df_scania: pd.DataFrame, weight_boost=1.2):
    if df_scania is None or df_scania.empty:
        return rules
    rules = rules.copy()
    def boost(row):
        ante = {str(x) for x in row["antecedents"]}
        if any(a.startswith("F_overheat") for a in ante):
            return row["score"] * weight_boost
        return row["score"]
    rules["score"] = rules.apply(boost, axis=1)
    return rules

rules_reweighted = apply_scania_weighting(rules, df_scania, weight_boost=1.2)
rules_reweighted = rules_reweighted.sort_values("score", ascending=False).reset_index(drop=True)
rules_reweighted.head(5)[["ante_str","cons_str","support","confidence","lift","score"]]


## Export rules & metrics

In [ ]:
rules_out = rules_reweighted.copy() if 'rules_reweighted' in globals() else rules.copy()
rules_out.to_csv("apriori_rules_parts_tech.csv", index=False)
eval_df.to_csv("evaluation_topk.csv", index=False)
print("Saved: apriori_rules_parts_tech.csv, evaluation_topk.csv")


## How to use on your data
1. Upload your CSVs to Colab (`/content/...`) or adjust the paths in **Configuration**.
2. Map your real columns in `CONFIG`:
   - `fault_category_col` (e.g., `PROB_TYPE`)
   - `device_type_col`, `part_col`, `technician_group_col` (if present)
   - If missing, keep them `None` and the notebook will **extract from `DESCRIPTION`** using the lexicons.
3. (Optional) Improve the lexicons/rules to better match your data vocabulary.
4. Tune `min_support`, `min_confidence`, and `min_lift` in `mine_rules(...)`.
5. Run evaluation; inspect `evaluation_topk.csv`. Review top rules in `apriori_rules_parts_tech.csv`.
6. For cross‑domain: make sure the `domain` column reflects 'vehicle'/'facility' vs. 'aviation', or edit `infer_domain(...)`.
